## Teste Data Science - Parte 2
### Raphael Ballet 
raphaelballet@gmail.com

---
### Objetivo: 
Criar um analisador de sentimentos de avaliações de filme obtidos pelo Rotten Tomatoes, que estão disponíveis no seguinte [*dataset*](http://www.cs.cornell.edu/people/pabo/movie-review-data/). O sistema deve classificar as avaliações entre "positivas" e "negativas", o que refletiria a opinião de cada avaliador sobre o filme.

---
### Solução:

Como apresentado na parte 1 desse trabalho, o processo de resolução desse problema será feito de maneira incremental e seguindo as etapas determinadas anteriormente. Para facilidade de referência, as etapas são resumidas a seguir:

- **Etapa 0**: Estudo preliminar do problema e definição do *framework* (foi apresentado na Parte 1)
- **Etapa 1**: Análise inicial do *dataset* e pré-processamento (limpeza) dos dados
- **Etapa 2**: Processamento dos dados e criação dos *bag-of-words*
- **Etapa 3**: Aplicação e comparação (via [validação cruzada *k-fold*](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) dos métodos de *machine learning* ([SVM](https://en.wikipedia.org/wiki/Support_vector_machine), [*Naive Bayes*](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) e [regressão logística](https://web.stanford.edu/~jurafsky/slp3/7.pdf)) para predição da classificação de sentimento
- **Etapa 4**: Validação final métodos de classificação binária utilizando os dados de teste
- **Etapa 5**: Conclusões sobre os métodos e possíveis problemas ou melhorias

---
### Etapa 1: Pré-Processamento dos Dados

O primeiro passo a ser tomado é a leitura dos arquivos de texto que contém as avaliações positivas e negativas. Para esse fim, utilizarei a biblioteca `pandas` do Python, que possui funções úteis para lidar com dados.

In [1]:
import pandas as pd

''' Leitura dos dados
    Considero que o arquivo apresenta apenas uma coluna, 
    que representa o texto completo da avaliação
'''
# Avaliação positiva:
pos_aval = pd.read_csv('rt-polarity.pos', header=None,  sep='\n', names=['avaliacao'])

# Avaliação negativa:
neg_aval = pd.read_csv('rt-polarity.neg', header=None,  sep='\n', names=['avaliacao'])

# Exemplos:
print('Positivo:\n',pos_aval['avaliacao'][0])
print('-----')
print('Negativo:\n',neg_aval['avaliacao'][0])

Positivo:
 the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 
-----
Negativo:
 simplistic , silly and tedious . 


Agora, precisamos indicar na tabela qual o sentimento das avaliações e unir as duas tabelas em uma única.

In [2]:
''' Avaliação do modelo
    Positiva: 1
    Negativa: 0
'''
pos_aval['sentimento'] = 1
neg_aval['sentimento'] = 0

# Concatenação das duas tabelas em apenas uma
aval = pd.concat([pos_aval, neg_aval])

# Embaralhamento aleatório das avaliações
aval = aval.sample(frac=1).reset_index(drop=True)

# Cria uma coluna com a cópia da avaliação original
aval['aval_orig'] = aval['avaliacao']

# Exemplos
aval.head(10)

,avaliacao,sentimento,aval_orig
0,the dragons are the real stars of reign of fir...,1,the dragons are the real stars of reign of fir...
1,[it's] a clever thriller with enough unexpecte...,1,[it's] a clever thriller with enough unexpecte...
2,"katz uses archival footage , horrifying docume...",1,"katz uses archival footage , horrifying docume..."
3,at its best early on as it plays the culture c...,1,at its best early on as it plays the culture c...
4,adam sandler's eight crazy nights grows on you...,0,adam sandler's eight crazy nights grows on you...
5,includes too much obvious padding .,0,includes too much obvious padding .
6,it may sound like a mere disease-of- the-week ...,1,it may sound like a mere disease-of- the-week ...
7,"the film feels uncomfortably real , its langua...",1,"the film feels uncomfortably real , its langua..."
8,"while the film is not entirely successful , it...",1,"while the film is not entirely successful , it..."
9,while easier to sit through than most of jaglo...,0,while easier to sit through than most of jaglo...


#### Limpeza dos dados

Como é possível ver nos exemplos anteriores, existem diversos problemas que precisamos arrumar. As pontuações não serão utilizadas nessa análise (apesar de poderem apresentar alguma relação com sentimento, como "!!!"). Outro fator que precisamos corrigir é a contração de palavras em inglês, como "isn't" ou "it's", ao invés de "is not" e "it is", respectivamente. Além disso, também retirarei números da análise (mais uma vez, números também poderiam refletir sentimento, como "this is 10/10!", mas foram retirados da análise para a simplificação).

Para essa tarefa de limpeza dos dados, utilizarei o método das [expressões regulares](https://en.wikipedia.org/wiki/Regular_expression).

In [3]:
# 1) Corrigir possíveis contrações negativas "isn't" -> "is not".
aval['avaliacao'].replace(to_replace="n't ", value = ' not ', inplace=True, regex=True)

# 2) Remover todas as pontuações e números
aval['avaliacao'].replace(to_replace='[^a-zA-Z]', value = ' ', inplace=True, regex=True)

# 3) Remover letras isoladas do texto que podem ter sido causadas pelas eliminações anteriores
aval['avaliacao'].replace(to_replace='(^| ).( |$)', value = ' ', inplace=True, regex=True)

# Exemplos
aval.head(10)

,avaliacao,sentimento,aval_orig
0,the dragons are the real stars of reign of fir...,1,the dragons are the real stars of reign of fir...
1,it clever thriller with enough unexpected tw...,1,[it's] a clever thriller with enough unexpecte...
2,katz uses archival footage horrifying document...,1,"katz uses archival footage , horrifying docume..."
3,at its best early on as it plays the culture c...,1,at its best early on as it plays the culture c...
4,adam sandler eight crazy nights grows on you ...,0,adam sandler's eight crazy nights grows on you...
5,includes too much obvious padding,0,includes too much obvious padding .
6,it may sound like mere disease of the week tv...,1,it may sound like a mere disease-of- the-week ...
7,the film feels uncomfortably real its language...,1,"the film feels uncomfortably real , its langua..."
8,while the film is not entirely successful it s...,1,"while the film is not entirely successful , it..."
9,while easier to sit through than most of jaglo...,0,while easier to sit through than most of jaglo...


Ao analisar os exemplos, podemos notar que o texto está consideravelmente mais limpo e pronto para ser processado. Essa etapa de pré-processamento reduz grande parte dos problemas que podem aparecer posteriormente se não corretamente endereçada.

---
### Etapa 2: Processamento dos Dados

A etapa de processamento dos dados é a responsável por criar os *bag-of-words* que serão utilizados no classificador binário de sentimentos. Essa etapa também será responsável por remover o que são chamados de [*stop-words*](https://en.wikipedia.org/wiki/Stop_words), que são palavras frequentes em qualquer documento escrito em uma dada linguagem. Em inglês, temos, por exemplo, os artigos definidos "the" e indefinidos {"a", "an"} que não possuem grande informação a respeito de um determinado texto.

Portanto, o primeiro passo será eliminar esses *stop-words* das avaliações. Isso será feito com a ajuda da biblioteca `nltk` ([Natural Language Toolkit](http://www.nltk.org/) do Python.

In [4]:
# Biblioteca NLTK
import nltk

# Ao utilizar pela primeira vez é necessário fazer o download do pacote de dados do NLTK
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Raphael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Obter os "stop-words"
from nltk.corpus import stopwords

# Exemplos de "stop-words"
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [6]:
# Separa o texto em uma lista de palavras
aval['avaliacao'] = aval['avaliacao'].str.split()

# Retira do texto todos os "stop-words"
aval['avaliacao'] = aval['avaliacao'].apply(lambda x: [i for i in x if i not in set(stopwords.words("english"))])

# Recoloca o texto em uma string
aval['avaliacao'] = aval['avaliacao'].str.join(sep=" ")

# Exemplos
aval.head(10)

,avaliacao,sentimento,aval_orig
0,dragons real stars reign fire wo disappointed,1,the dragons are the real stars of reign of fir...
1,clever thriller enough unexpected twists keep ...,1,[it's] a clever thriller with enough unexpecte...
2,katz uses archival footage horrifying document...,1,"katz uses archival footage , horrifying docume..."
3,best early plays culture clashes brothers,1,at its best early on as it plays the culture c...
4,adam sandler eight crazy nights grows like rash,0,adam sandler's eight crazy nights grows on you...
5,includes much obvious padding,0,includes too much obvious padding .
6,may sound like mere disease week tv movie song...,1,it may sound like a mere disease-of- the-week ...
7,film feels uncomfortably real language locatio...,1,"the film feels uncomfortably real , its langua..."
8,film entirely successful still manages string ...,1,"while the film is not entirely successful , it..."
9,easier sit jaglom self conscious gratingly irr...,0,while easier to sit through than most of jaglo...


#### Bag-of-words

Agora é possível criar o *bag-of-words*. Utilizarei a biblioteca `scikit-learn` do Python para realizar essa tarefa. A função [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) permite criar uma representação esparsa da frequência que cada palavra em uma frase representa no contexto geral das avaliações. Para esse trabalho, vou utilizar *bag-of-words* com *unigrams* e *bigrams*, onde sua frequência de contagem será normalizada utilizando [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), com a função [`TfidfTransformer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer). Dessa forma, podemos evitar dar maior peso a palavras muito frequentes no texto que não dão grande valor ao conteúdo da avaliação.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
'''
    - CountVectorizer: Cria uma representação esparsa de palavras (features) de cada linha da tabela de 
    avaliações em relação à todas as palavras contidas na tabela. 
    - Opções: analyzer = "word" -> Indica que as features são palavras
              ngram_range = (min, max) -> Indica o valor mínimo e máximo de "n-grams" que utilizaremos
                                          Ex: Para (1,2), a frase "dog cat" conteria tanto os unigramas
                                          {dog} e {cat}, quanto o bigrama {dog cat}.
              max_features = N -> Indica o valor máximo de "features" que vamos considerar.
                                  O algoritmo escolhe as N palavras mais relevantes no texto.
              binary -> Se False, a matriz esparsa contem a contagem de vezes que uma dada palavra é
                        obtida em uma única avaliação. Se True, a matriz apenas indica a presença ou
                        ausência da palavra na avaliação.
'''
# Obtém o objeto que será utilizado para criar o bag-of-words
vect = CountVectorizer(analyzer = "word", ngram_range = (1, 2), max_features = 10000, binary = False)

# Obtém o objeto que será utilizado para transformar o bag-of-words utilizando tf-idf
transf = TfidfTransformer()

# Cria o bag-of-words
X = vect.fit_transform(aval['avaliacao']).toarray()
# Exemplo
print("Exemplo contagem absoluta: \n", X[X>0][:10])

# Transforma tf-idf (Novo bag-of-words (bow))
aval_bow = transf.fit_transform(X).toarray()

# Exemplo: Mostra apenas alguns valores positivos com os dois métodos
print("\nExemplo contagem absoluta: \n", aval_bow[aval_bow>0][:10])

Exemplo contagem absoluta: 
 [1 1 1 1 1 1 1 1 1 1]

Exemplo contagem absoluta: 
 [ 0.35616728  0.36932104  0.30593513  0.24129645  0.34596442  0.349119
  0.31427147  0.28182274  0.40639942  0.37097615]


Agora, podemos separar os dados entre dados de treino e dados de teste. Como apresentado na Parte 1 deste trabalho, os dados serão separados em 80/20, ou seja 80% dos dados originais serão utilizados para treinamento e 20% dos dados serão utilizados para teste.

In [8]:
import numpy as np

# Porcentagem de treinamento: 80%
pct_treino = 0.8

# Tamanho do dataset
tam_dados = aval_bow.shape[0]

# Separa dados
# Treino
x_treino = aval_bow[:int(pct_treino*tam_dados)]
y_treino = np.array(aval['sentimento'])[:int(pct_treino*tam_dados)]

# Teste
x_teste = aval_bow[int(pct_treino*tam_dados):]
y_teste = np.array(aval['sentimento'])[int(pct_treino*tam_dados):]

# Mostra a quantidade de dados em cada um deles
print(x_treino.shape, x_teste.shape)

(8529, 10000) (2133, 10000)


---
### Etapa 3: Classificadores

Estamos prontos para começar o treinamento! Nessa etapa o principal objetivo é implementar os métodos de *machine learning* para treinar o classificador de sentimentos. Na etapa anterior, os dados foram preparados entre dados de treino e dados de teste. Nessa etapa só serão utilizados os dados de treino. Na etapa posterior, os dados de teste serão utilizados para a obtenção final da acurácia de cada método.

Para aplicar os métodos, utilizarei novamente a biblioteca `scikit-learn` do Python. Portanto, primeiramente será necessário importar as funções para o *notebook*.

Os algoritmos que serão comparados são:

- [**Regressão Logística**](https://web.stanford.edu/~jurafsky/slp3/7.pdf) (RL): Método clássico para classificação categórica. O objetivo é encontrar os coeficientes das variáveis independentes de um hiperplano que separa linearmente os dados de uma categoria. A regressão logística então utiliza a [função logística](https://en.wikipedia.org/wiki/Logistic_function) (ou logit) da saída predita para gerar um número contínuo entre [0, 1], que tem a propriedade de indicar a probabilidade de um dado ponto de entrada ser classificado de uma determinada maneira. Comumente, adota-se que qualquer valor > 0.5 é caracterizado como 1 (ou positivo, no caso da classificação de filmes) e, se for abaixo de 0.5, o dado é caracterizado como 0 (ou negativo). Entretanto, o valor limítrofe pode sofrer alteração conforme necessidade (por exemplo, reduzir erros do [tipo 1 ou 2](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors));

In [9]:
from sklearn.linear_model import LogisticRegression

- [**Naive Bayes**](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) ([*Multinomial Naive Bayes*](http://scikit-learn.org/stable/modules/naive_bayes.html#multinomial-naive-bayes)): O método *Naive Bayes* (NB) é comumente utilizado para problemas de análise de sentimento em texto (veja [aqui](https://web.stanford.edu/~jurafsky/slp3/6.pdf)), gerando bons resultados. Esse método utiliza o [teorema de Bayes](https://en.wikipedia.org/wiki/Bayes%27_theorem) para calcular a probabilidade *a posteriori* de uma saída, dadas as medidas dos componentes de entrada e suas probabilidades *a priori*. Esse problema é de difícil resolução analítica para o caso completo, mas se torna simples se considerarmos que as distribuições do conjunto de variáveis de entrada são (mutuamente) idependentes. Dessa forma, podemos facilmente calcular a variável de saída ao maximizar a probabilidade *a posteriori* (também conhecido como estimador [MAP](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation)) do modelo. Entretanto, existem variações do algoritmo NB, que contemplam diferentes tipos de suposições sobre a distribuição das variáveis de entrada. No caso de análise de sentimentos utilizando *bag-of-words*, normalmente é utilizado a [distribuição multinomial](https://en.wikipedia.org/wiki/Multinomial_distribution), que apresenta características úteis para a distribuição da análise de frequência das palavras no texto.

In [10]:
from sklearn.naive_bayes import MultinomialNB

- [**SVM**](https://en.wikipedia.org/wiki/Support_vector_machine): Outro método normalmente utilizado para análise de sentimento de texto é o *Support Vector Machine*. No caso de classificação binária, o algoritmo SVM procura criar um hiperplano no qual a fronteira entre os dados de classes distintas é obtida a partir de métodos de otimização. Após encontrar o hiperplano, os novos dados são classificados binariamente dependendo da sua localização em relação ao hiperplano. O SVM é normalmente utilizado para problemas linearmente separáveis, mas também pode ser modificado para sistemas não lineares ao modificar o seu [*kernel*](https://en.wikipedia.org/wiki/Kernel_method).

In [11]:
from sklearn.svm import LinearSVC

O próximo passo é criar um grupo de classificadores que serão utilizados para a comparação.

In [12]:
classificadores = {'Regressão Logistica': LogisticRegression(),
                   'Naive Bayes': MultinomialNB(),
                   'SVM': LinearSVC() }

Utiliza-se a [validação cruzada *k-fold*](https://en.wikipedia.org/wiki/Cross-validation_(statistics) para comparar a acurácia dos classificadores. A validação cruzada é um ótimo jeito de evitar que os dados utilizados para treino e validação contenham "vícios" ou vieses. Para que isso seja feito, dividimos aleatoriamente o conjunto de dados em *k* grupos distintos, onde *k*-1 grupos são utilizados para treino e 1 grupo é utilizado para a validação. Repete-se esse processo por *k* vezes, obtendo *k* valores de alguma das métricas que podem ser utilizadas para avaliar a qualidade do modelo (adotam-se, principalmente, as métricas de acurácia, precisão, [*recall*](https://en.wikipedia.org/wiki/Precision_and_recall), [*f1-score*](https://en.wikipedia.org/wiki/F1_score) etc.)

Nesse trabalho, adota-se a validação cruzada utilizado *k* = 10 e o métrica utilizada é a acurácia. 

*Obs: Nesse caso, considero que a avaliação de falsos positivos (erro tipo 1) e de falsos negativos (erro tipo 2) têm o mesmo valor para o classificador. Se houvesse uma preferência para evitar algum dos tipos dos erros, seria necessário utilizar métodos mais informativos para a análise dos modelos, como analisar as curvas [ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) ou [matriz de confusão](https://en.wikipedia.org/wiki/Confusion_matrix).*

In [13]:
# Importar métodos de seleção de modelos do scikit-learn
from sklearn import model_selection
# Importar o método de avaliação de acurácia
from sklearn.metrics import accuracy_score

# Para cada modelo de classificador -> Realizar treinamento
# e validação cruzada utilizando k-fold
print("Modelo: Acuracia (Precisao)")
print("---------------------------")
for clas in classificadores:
    # Classificador
    modelo = classificadores[clas]
    # Criação dos grupos de validação cruzada
    kfold = model_selection.KFold(n_splits = 10)
    # Executa o treinamento e realiza a validação cruzada
    resultado = model_selection.cross_val_score(modelo, x_treino, y_treino, 
                                                scoring = 'accuracy', cv = kfold, n_jobs = 4)
    # Obtém as estatísticas do resultado (média e desvio padrão -> acurácia e precisão)
    media = resultado.mean()
    dp = resultado.std()
    
    # Apresenta os resultados dos classificadores
    print("{}: {:.4f} ({:.4f})".format(clas, media, dp))   

Modelo: Acuracia (Precisao)
---------------------------
SVM: 0.7506 (0.0154)
Naive Bayes: 0.7691 (0.0121)
Regressão Logistica: 0.7552 (0.0188)


Como pode ser visto, o classificador NB apresentou a maior acurácia (76,91 %) ao classificar o sentimento do texto utilizando os dados de treinamento em uma validação cruzada *k-fold*. Interessantemente, o método SVM apresentou o pior desempenho em comparação ao NB e Regressão Logística. Esse resultado é diferente do encontrado em outros artigos [[1]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf) [[2]](http://www.cs.cornell.edu/home/llee/papers/cutsent.pdf). O SVM apresentou acurácia de 75,06%, enquanto o modelo de Regressão Logística apresentou acurácia de 75,52%.

---
### Etapa 4: Validação

O próximo passo é analisar o comportamento dos classificadores ao predizer o sentimento de dados que não foram usados no treinamento. Espera-se que a acurácia do modelo seja semelhante a obtida pela validação cruzada do item anterior. O conjunto de dados de teste se referem a 20% do total do *dataset* inicial.

In [14]:
# Dados de teste: x_teste, y_teste

# Para cada modelo de classificador -> Predizer classificação
print("Modelo: Acuracia")
print("---------------------------")
for clas in classificadores:
    # Classificador
    modelo = classificadores[clas]
    # Executa o treinamento do modelo
    modelo.fit(x_treino, y_treino)
    # Obtém as predições do classificador
    y_pred = modelo.predict(x_teste)
    # Calcula a acurácia do modelo
    acuracia = accuracy_score(y_teste, y_pred)
    # Apresenta os resultados dos classificadores
    print("{}: {:.4f}".format(clas, acuracia))

Modelo: Acuracia
---------------------------
SVM: 0.7543
Naive Bayes: 0.7829
Regressão Logistica: 0.7707


Novamente, o método NB apresentou maior acurácia em relação ao SVM e Regressão Logística. O resultado de 78,29% é um resultado importante, já que espera-se que até mesmo humanos possuam acurácia próxima a 80% [[3]](http://mashable.com/2010/04/19/sentiment-analysis/#KrPG2rUv45qh).

---
### Etapa 5: Conclusão

O seguinte trabalho teve como objetivo obter um classificador de sentimentos a partir de avaliações de filmes obtidos pelo Rotten Tomatoes. O trabalho foi apresentado em 5 etapas (6, se contar a etapa de estudo preliminar) de desenvolvimento, na qual cada etapa foi explicada e demonstrada a partir de um contexto lógico. As etapas 1 e 2 tiveram o objetivo de preparar os dados brutos para serem utilizados no treinamento dos classificadores. Para esse fim, diversas simplificações e suposições lógicas foram feitas no texto, como a retirada de pontuações, contrações e *stop-words*. A etapa 3 apresentou a comparação dos classificadores escolhidos para resolver o problema proposto. Outros classificadores poderiam ser escolhidos para essa análise, mas procurou-se utilizar métodos comumente encontrados na literatura, como o SVM e Naive Bayes, e também um método clássico para classificação binária, que é a Regressão Logística. Os métodos foram comparados utilizando validação cruzada *k-fold*, que permitiu constatar que o classificador Naive Bayes apresentou a melhor acurácia para classificar o sentimento dos filmes, seguido da Regressão Logística e do SVM. Portanto, é possível concluir que o trabalho atual atingiu os objetivos inicialmente propostos e gerou um estudo relevante para a análise do problema. Espera-se que o presente estudo seja ampliado futuramente com a adoção de outros métodos de classificação, assim como outras técnicas de parametrização do texto.

### Referências

[[1]](http://www.cs.cornell.edu/home/llee/papers/sentiment.pdf) Pang, B., Lee, L. e Vaithyanathan, S. "Thumbs up? Sentiment Classification using Machine Learning Techniques", Proceedings of EMNLP, 2002;

[[2]](http://www.cs.cornell.edu/home/llee/papers/cutsent.pdf) Pang, B. e Lee, L. "A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts", Proceedings of ACL, 2004;

[[3]](http://mashable.com/2010/04/19/sentiment-analysis/#KrPG2rUv45qh) Ogneva, M. "How Companies Can Use Sentiment Analysis to Improve Their Business", Mashable: http://mashable.com/2010/04/19/sentiment-analysis/#KrPG2rUv45qh, 2010;